# GraphRAG-图数据库操作

- pip install py2neo

In [1]:
from py2neo import Graph, Node, Relationship

In [2]:
import json
from tqdm import tqdm
from pprint import pprint

## 连接到Neo4j数据库

In [3]:
graph = Graph("bolt://180.xx.26.xx:7687", user='neo4j', password='neo4j@123',name='neo4j')

## 解析数据

- 公司事件提取数据

In [4]:
event_type_file = './data/iree.json'
event_type_data = []
with open(event_type_file, 'r', encoding='utf-8') as file:
    for line in file.readlines():
        event_type_data.append(json.loads(line.strip()))

In [5]:
pprint(event_type_data[20], sort_dicts=False)

{'title': '雪佛龙计划大规模裁员 约6000个工作岗位面临风险',
 'content': '北京时间28日消息，雪佛龙计划今年将全球员工人数削减10%至15%，这是在新冠肺炎之后全球石油巨头中最大规模的裁员。此番裁员相当于该公司45，000名非加油站员工中的约6，000人，并可能是BP和壳牌等大型石油公司裁员的先兆。截至目前，裁员主要集中在油田服务领域和北美的独立生产商之中。雪佛龙“正在简化我们的组织结构，以反映出效率并与预期的活动水平保持一致，”该公司周三在一份公告中说。“这是一个艰难的决定，我们不会轻易做出这样的决定。”',
 'label': [{'event_type': '裁员',
            'arguments': [{'主体': '雪佛龙'}, {'时间': '北京时间28日'}]}]}


In [6]:
pprint(event_type_data[30], sort_dicts=False)

{'title': '科伦药业：控股股东解除1100万股质押再质押1300万股',
 'content': '5月26日晚间，科伦药业发布公告称，公司董事长、控股股东刘革新于2020年5月22日质押1300万股，占其所持股份比例为3.43%。2020年5月25日，刘革新将原质押给华泰证券（上海）资产管理有限公司的公司股份1100万股解除质押，占其所持股份比例为2.9%。截至本公告日，刘革新累计质押股数约为2.08亿股，占其所持股份比例为54.78%。',
 'label': [{'event_type': '股权质押',
            'arguments': [{'时间': '2020年5月25日'},
                          {'数值': ['1100万股']},
                          {'主体': '科伦药业'}]}]}


- 投资数据

In [7]:
invest_file = './data/invest-on-invent-KG.json'
with open(invest_file, 'r', encoding='utf-8') as file:
    invest_data = json.load(file)


In [8]:
invest_data['@graph'][0]

{'@id': '0',
 '@type': 'investor',
 'name': '瑞华林投资',
 'relationship': {'investCompany': [{'@id': '5617',
    '@type': 'company',
    'round': '新三板定增',
    'date': '2016-03-04'}]}}

In [9]:
invest_data['@graph'][6382]

{'@id': '6382',
 '@type': 'company',
 'name': '广州中设机器人智能装备股份有限公司',
 'alterNames': ['中设机器人智能装备',
  '中设机器人智能装备公司',
  '广州中设机器人智能装备',
  '广州市中设机器人智能装备',
  '广州中设机器人智能装备公司',
  '广州市中设机器人智能装备公司'],
 'establishDate': '2008-07-23',
 'address': '广州市黄埔区云埔工业区方达路6号101房',
 'relationship': {'applyPatent': []}}

In [10]:
invest_data['@graph'][141764]

{'@id': '141764',
 '@type': 'patent',
 'name': '一种基于DLL的软件架构',
 'patentType': '发明公布'}

## 数据处理
   - 投资数据中：投资者和投资公司 公司和专利 id的对应关系
   - 投资数据的公司名称和事件数据名称：名称的对应关系

In [11]:
obj_by_id = {}

for ele in invest_data["@graph"]:
    obj_by_id[ele["@id"]] = ele

In [12]:
ignore_names = ['电器', '600万股', '公司', '135万元', '2018年', '新产业']
even_type_company_names = {}
for data in event_type_data:
    for label in data['label']:
        if 'arguments' in label:
            for arg in label['arguments']:
                for key, value in arg.items():
                    if key == "主体":
                        if value in ignore_names:
                            continue
                        even_type_company_names[value] = 1

In [13]:
list(even_type_company_names.keys())[:10]

['小米集团',
 '小米',
 '格力电器',
 '波音公司',
 '波音',
 'GoPro',
 '得邦照明股份有限公司',
 '得邦照明',
 '横店集团得邦工程塑料有限公司',
 '汉邦高科']

In [14]:
invest_company_names = []

for ele in invest_data["@graph"]:
    if ele["@type"] == "company":
        invest_company_names.append(ele["name"])

In [15]:
invest_company_names[:10]

['广州博鳌纵横网络科技有限公司',
 '福建博思软件股份有限公司',
 '深圳优地科技有限公司',
 '杭州群核信息技术有限公司',
 '昆山希盟自动化科技有限公司',
 '北京小爱智能科技有限公司',
 '奇安信科技集团股份有限公司',
 '北京大米科技有限公司',
 '北京诸葛找房信息技术有限公司',
 '苏州奥易克斯汽车电子有限公司']

In [16]:
even_type_company_dict = {}
for company in even_type_company_names.keys():
    for iv_cname in invest_company_names:
        if company in iv_cname:
            even_type_company_dict.setdefault(company, []).append(iv_cname)

In [17]:
even_type_company_dict['小米']

['小米科技有限责任公司']

In [18]:
even_type_company_dict['格力电器']

['珠海格力电器股份有限公司']

In [19]:
even_type_company_dict['瑞幸咖啡']

['瑞幸咖啡(北京)有限公司']

## 构建节点和边
- 公司和事件
- 公司和投资者
- 公司和专利

In [ ]:
for data in tqdm(event_type_data):
    title = data['title']
    content = data['content']
    
    for label in data['label']:
        event_type = label['event_type']
        event_type_node = Node("EventType", name=event_type)
        graph.merge(event_type_node, "EventType", "name")
        
        if 'arguments' in label:
            
            cname = ''
            tmp_args = {}
            for arg in label['arguments']:
                for key, value in arg.items():
                    if key == "主体":
                        cname = value
                    else:
                        tmp_args[key] = value
            if cname in even_type_company_dict:
                for company_name in even_type_company_dict[cname]:
                    company_node = Node("Company", name=company_name)
                    graph.merge(company_node, "Company", "name")
                    
                    relationship = Relationship(company_node, "HAPPEN", event_type_node)
                    for key, value in tmp_args.items():
                        relationship[key] = value
                    
                    relationship['title'] = title
                    relationship['content'] = content
                    graph.create(relationship)

In [ ]:
for ele in tqdm(invest_data["@graph"]):
    if ele["@type"] == "company":
        name = ele["name"]
        alterNames = ele["alterNames"]
        establishDate = ele["establishDate"]
        address = ele["address"]
        applyPatent = ele["relationship"]["applyPatent"]
        
        company_node = Node("Company", name=name, establishDate=establishDate, address=address)
        graph.merge(company_node, "Company", "name")
        
        if len(applyPatent):
            for patent in applyPatent:
                patent_name = obj_by_id[patent['@id']]['name']
                patent_node = Node("Patent", name=patent_name)
                graph.merge(patent_node, "Patent", "name")
                
                relationship = Relationship(company_node, "HAVE", patent_node)
                graph.create(relationship)
    elif ele["@type"] == "investor":
        investor_node = Node("Investor", name=ele['name'])
        graph.merge(investor_node, "Investor", "name")
        
        invest_company = ele["relationship"]["investCompany"]
        for invest in invest_company:
            acompany_ele = obj_by_id[invest['@id']]
            name = acompany_ele["name"]
            alterNames = acompany_ele["alterNames"]
            establishDate = acompany_ele["establishDate"]
            address = acompany_ele["address"]
            applyPatent = acompany_ele["relationship"]["applyPatent"]
            
            company_node = Node("Company", name=name, establishDate=establishDate, address=address)
            graph.merge(company_node, "Company", "name")
            
            relationship = Relationship(investor_node, "INVEST", company_node)
            relationship['date'] = invest['date']
            relationship['round'] = invest['round']
            graph.create(relationship)
            
        

## MATCH查询

In [20]:
query = """
MATCH (company)-[r:HAPPEN]->(e:EventType {name: "业绩承诺未达标"})
RETURN company.name as name,r.title as title
"""

# 执行查询
results = graph.run(query)
for record in results:
    print(dict(record))

{'name': '北京真视通科技股份有限公司', 'title': ''}
{'name': '上海中锂实业有限公司', 'title': ''}
{'name': '浙江南都电源动力股份有限公司', 'title': ''}
{'name': '上海科匠信息科技有限公司', 'title': ''}
{'name': '安徽晶奇网络科技股份有限公司', 'title': ''}
{'name': '阳光城集团股份有限公司', 'title': ''}
{'name': '浙江聚力文化发展股份有限公司', 'title': ''}
{'name': '青岛万达影视投资有限公司', 'title': '商誉暴雷，业绩亏损，万达电影缘何还能逆势扩张'}
{'name': '浙江田中精机股份有限公司', 'title': '0.3折“卖子”给大股东 田中精机被疑利益输送'}
{'name': '高斯贝尔数码科技股份有限公司', 'title': '高斯贝尔持续经营能力遭深交所质疑 子公司业绩承诺期业绩均未'}
{'name': '中文在线数字出版集团股份有限公司', 'title': '市场动态 监管动态 | 收购子公司未达业绩承诺，中文在线收北京证监局警示函'}
{'name': '广东东方精工科技股份有限公司', 'title': '卷进东方精工业绩纠纷，宁德时代称对方的披露严重失实'}
{'name': '鄂州二医院有限公司', 'title': '上市公司医院投资项目暴雷 医院业绩暴跌'}
{'name': '博瑞生物医药(苏州)股份有限公司', 'title': '博瑞生物签订4项对赌协议被追问 \xa0 业绩承诺连续三年未完成'}
{'name': '云南龙发制药股份有限公司', 'title': '一份对赌协议引发的风险！融资7200万元，这家新三板公司从计划IPO到面临“易主”'}
{'name': '北京汉邦高科数字技术股份有限公司', 'title': '重组标的业绩承诺未兑现，汉邦高科将回购股份作抵消'}


In [21]:
query = """
MATCH (c:Company)-[r:HAPPEN]->(e)
WHERE c.name CONTAINS "比亚迪"
RETURN c.name as name, r.title as title, e.name as even
"""
results = graph.run(query)
for record in results:
    print(dict(record))

{'name': '比亚迪股份有限公司', 'title': '比亚迪上半年新增借款138.7亿 连续三年新增借款超上年净资产20％', 'even': '债务增加'}
{'name': '深圳比亚迪电动汽车投资有限公司', 'title': '比亚迪上半年新增借款138.7亿 连续三年新增借款超上年净资产20％', 'even': '债务增加'}
{'name': '比亚迪股份有限公司', 'title': '方向错了？动力电池市场将再掀起磷酸铁锂热潮', 'even': '产品创新'}
{'name': '深圳比亚迪电动汽车投资有限公司', 'title': '方向错了？动力电池市场将再掀起磷酸铁锂热潮', 'even': '产品创新'}
{'name': '比亚迪股份有限公司', 'title': '车界：新能源车，成也补贴，败也补贴', 'even': '利润下滑'}
{'name': '深圳比亚迪电动汽车投资有限公司', 'title': '车界：新能源车，成也补贴，败也补贴', 'even': '利润下滑'}
{'name': '比亚迪股份有限公司', 'title': '比亚迪(01211.HK)与日野汽车合作研发纯电动商用车', 'even': '业务合作'}
{'name': '深圳比亚迪电动汽车投资有限公司', 'title': '比亚迪(01211.HK)与日野汽车合作研发纯电动商用车', 'even': '业务合作'}
{'name': '比亚迪股份有限公司', 'title': '好事连连', 'even': '引进战略投资'}
{'name': '深圳比亚迪电动汽车投资有限公司', 'title': '好事连连', 'even': '引进战略投资'}
{'name': '比亚迪股份有限公司', 'title': '比亚迪不单单是家车企，这4个副业个个有名，难怪能成领头羊！', 'even': '企业转型'}
{'name': '深圳比亚迪电动汽车投资有限公司', 'title': '比亚迪不单单是家车企，这4个副业个个有名，难怪能成领头羊！', 'even': '企业转型'}


In [22]:
query = """
MATCH (iv:Investor)-[i:INVEST]->(c:Company {name: "科大讯飞股份有限公司"})
RETURN iv.name
"""

results = graph.run(query)
for record in results:
    print(dict(record))

{'iv.name': '方德信'}
{'iv.name': '嘉信元德股权投资基金'}
{'iv.name': '东方星空投资'}
{'iv.name': '诚禧投资'}
{'iv.name': '上海合鲸乐宜投顾'}
{'iv.name': '廊坊冀财新毅创业引导股权投资基金(有限合伙)'}
{'iv.name': '红十月投资'}
{'iv.name': '深圳诚成高科'}
{'iv.name': '贵州东方世纪投资管理企业(有限合伙)'}
{'iv.name': '聚丰博和'}
{'iv.name': '尚珹资本'}
{'iv.name': '中瑞深圳'}


In [27]:
query = """
MATCH (iv:Investor {name: "中瑞深圳"})-[i:INVEST]->(c)-[:HAPPEN]->(e)
RETURN iv.name,c.name,e.name
"""

# 执行查询
results = graph.run(query)
for record in results:
    print(dict(record))

{'iv.name': '中瑞深圳', 'c.name': '科大讯飞股份有限公司', 'e.name': '裁员'}
{'iv.name': '中瑞深圳', 'c.name': '科大讯飞股份有限公司', 'e.name': '产品创新'}
{'iv.name': '中瑞深圳', 'c.name': '科大讯飞股份有限公司', 'e.name': '股份解禁'}


In [28]:
query = """
MATCH (other)<-[:INVEST]-(iv:Investor)-[:INVEST]->(c:Company {name: "杭州格像科技有限公司"})
RETURN iv.name, other.name
"""

# 执行查询
results = graph.run(query)
for record in results:
    print(dict(record))

{'iv.name': '羽信资本', 'other.name': '上海妙克信息科技有限公司'}
{'iv.name': '羽信资本', 'other.name': '西安瑜乐文化科技股份有限公司'}
{'iv.name': '羽信资本', 'other.name': '深圳市茁壮网络股份有限公司'}
{'iv.name': '广州基金', 'other.name': '苏州朗动网络科技有限公司'}
